In [1]:
!wget https://www.labri.fr/perso/lsimon/option-ia/Alphago/samples-9x9.json.gz

--2020-11-13 09:25:27--  https://www.labri.fr/perso/lsimon/option-ia/Alphago/samples-9x9.json.gz
Resolving www.labri.fr (www.labri.fr)... 147.210.8.59, 2001:660:6101:404::80
Connecting to www.labri.fr (www.labri.fr)|147.210.8.59|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3185947 (3.0M) [application/x-gzip]
Saving to: ‘samples-9x9.json.gz.1’

samples-9x9.json.gz 100%[===================>]   3.04M  3.88MB/s    in 0.8s    

2020-11-13 09:25:29 (3.88 MB/s) - ‘samples-9x9.json.gz.1’ saved [3185947/3185947]



In [34]:
import gzip, json

with gzip.open("samples-9x9.json.gz","r") as fz:
  rawdata = json.loads(fz.read().decode("utf-8"))

print(len(rawdata), "examples")
print("Exemple 0: ", rawdata[0])

41563 examples
Exemple 0:  {'depth': 41, 'list_of_moves': ['C6', 'G3', 'D3', 'E7', 'F3', 'G4', 'G7', 'C8', 'F6', 'B6', 'B5', 'E6', 'D5', 'F8', 'H6', 'F2', 'E2', 'H2', 'F5', 'F4', 'E3', 'B7', 'E5', 'G8', 'H8', 'A5', 'G9', 'B4', 'C5', 'H5', 'G5', 'C2', 'H4', 'C7', 'F9', 'E9', 'H9', 'C4', 'F1', 'C3', 'F7'], 'black_stones': ['F9', 'G9', 'H9', 'H8', 'F7', 'G7', 'C6', 'F6', 'H6', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H4', 'D3', 'E3', 'F3', 'E2', 'F1'], 'white_stones': ['E9', 'C8', 'F8', 'G8', 'B7', 'C7', 'E7', 'B6', 'E6', 'A5', 'H5', 'B4', 'C4', 'F4', 'G4', 'C3', 'G3', 'C2', 'F2', 'H2'], 'rollouts': 100, 'black_wins': 84, 'black_points': 1034.0, 'white_wins': 16, 'white_points': 296.0}


In [35]:
import numpy as np
from collections import namedtuple

Data = namedtuple('Data',['board','wins'])

def toCoord(s):
  c = ord('I' if s[0]=='J' else s[0]) - ord('A') # Bricolage parce que le I n'existe pas en GO
  l = int(s[1]) - 1
  return c, l

def arrangeSample(s):
  grid = np.zeros((9,9), dtype="int")
  for c in s["black_stones"]:
    cc = toCoord(c)
    assert 0 <= cc[0] < 9
    assert 0 <= cc[1] < 9
    grid[cc[0], cc[1]] = 1
  for c in s["white_stones"]:
    cc = toCoord(c)
    grid[cc[0], cc[1]] = 2
  return Data(grid, s["black_wins"])

gridtoencode = rawdata[0]
print("Exemple de donnée à prédire")
print("blacks", [(x, toCoord(x)) for x in gridtoencode["black_stones"]])
print("whites", gridtoencode["white_stones"])
print("Score to predict", gridtoencode["black_wins"])
print(arrangeSample(gridtoencode))

datas = [arrangeSample(x) for x in rawdata]


Exemple de donnée à prédire
blacks [('F9', (5, 8)), ('G9', (6, 8)), ('H9', (7, 8)), ('H8', (7, 7)), ('F7', (5, 6)), ('G7', (6, 6)), ('C6', (2, 5)), ('F6', (5, 5)), ('H6', (7, 5)), ('B5', (1, 4)), ('C5', (2, 4)), ('D5', (3, 4)), ('E5', (4, 4)), ('F5', (5, 4)), ('G5', (6, 4)), ('H4', (7, 3)), ('D3', (3, 2)), ('E3', (4, 2)), ('F3', (5, 2)), ('E2', (4, 1)), ('F1', (5, 0))]
whites ['E9', 'C8', 'F8', 'G8', 'B7', 'C7', 'E7', 'B6', 'E6', 'A5', 'H5', 'B4', 'C4', 'F4', 'G4', 'C3', 'G3', 'C2', 'F2', 'H2']
Score to predict 84
Data(board=array([[0, 0, 0, 0, 2, 0, 0, 0, 0],
       [0, 0, 0, 2, 1, 2, 2, 0, 0],
       [0, 2, 2, 2, 1, 1, 2, 2, 0],
       [0, 0, 1, 0, 1, 0, 0, 0, 0],
       [0, 1, 1, 0, 1, 2, 2, 0, 2],
       [1, 2, 1, 2, 1, 1, 1, 2, 1],
       [0, 0, 2, 2, 1, 0, 1, 2, 1],
       [0, 2, 0, 1, 2, 1, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]]), wins=84)
